In [1]:
import os

In [2]:
%pwd

'/Users/divyanshu9871gmail.com/Desktop/ml-ops-holiday-package-prediction/notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/divyanshu9871gmail.com/Desktop/ml-ops-holiday-package-prediction'

In [5]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv("./artifacts/data_validation/travel_processed.csv")
df.head()

,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalVisiting
0,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,0,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,0,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,0,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [8]:
X = df.drop("ProdTaken", axis=1)
y = df["ProdTaken"]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

In [10]:
cat_features = X.select_dtypes(include="object").columns
num_features = X.select_dtypes(exclude="object").columns

In [11]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop="first")

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [12]:
cat_features, num_features

(Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
        'MaritalStatus', 'Designation'],
       dtype='object'),
 Index(['Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups',
        'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
        'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'TotalVisiting'],
       dtype='object'))

In [14]:
df.apply(lambda col: col.unique())

ProdTaken                                                            [1, 0]
Age                       [41.0, 49.0, 37.0, 33.0, 36.0, 32.0, 59.0, 30....
TypeofContact                               [Self Enquiry, Company Invited]
CityTier                                                          [3, 1, 2]
DurationOfPitch           [6.0, 14.0, 8.0, 9.0, 30.0, 29.0, 33.0, 22.0, ...
Occupation                [Salaried, Free Lancer, Small Business, Large ...
Gender                                                       [Female, Male]
NumberOfFollowups                            [3.0, 4.0, 2.0, 5.0, 1.0, 6.0]
ProductPitched                [Deluxe, Basic, Standard, Super Deluxe, King]
PreferredPropertyStar                                       [3.0, 4.0, 5.0]
MaritalStatus                                [Unmarried, Divorced, Married]
NumberOfTrips             [1.0, 2.0, 7.0, 5.0, 6.0, 3.0, 4.0, 19.0, 21.0...
Passport                                                             [1, 0]
PitchSatisfa

In [15]:
df["Occupation"].value_counts()

Occupation
Salaried          2368
Small Business    2084
Large Business     434
Free Lancer          2
Name: count, dtype: int64

In [16]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [18]:
X_train

array([[ 1.        ,  0.        ,  1.        , ...,  0.78143477,
         0.4603181 , -0.06786916],
       [ 1.        ,  1.        ,  0.        , ..., -1.27969734,
        -1.00566841, -0.777901  ],
       [ 1.        ,  0.        ,  1.        , ...,  0.78143477,
        -1.2097217 , -0.06786916],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.78143477,
        -0.35739958,  0.64216267],
       [ 1.        ,  0.        ,  0.        , ...,  0.78143477,
        -0.25076528,  0.64216267],
       [ 0.        ,  0.        ,  1.        , ..., -1.27969734,
        -1.07168012, -1.48793284]])

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    processed_data_path: Path
    preprocessor_model_name: str

In [26]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        model_config_filepath=MODEL_CONFIG_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.model_config = read_yaml(model_config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        return DataTransformationConfig(
            root_dir=config.root_dir,
            processed_data_path=config.processed_data_path,
            preprocessor_model_name=config.preprocessor_model_name,
        )

In [28]:
from src.logger import logger
from sklearn.model_selection import train_test_split
import joblib

In [37]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.processed_data_path)

        train, test = train_test_split(data, test_size=0.30, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

    def fit_and_save_column_transformer(self):
        train = pd.read_csv(os.path.join(self.config.root_dir, "train.csv"))

        train_X = train.drop("ProdTaken", axis=1)

        cat_features = train_X.select_dtypes(include="object").columns
        num_features = train_X.select_dtypes(exclude="object").columns

        numeric_transformer = StandardScaler()
        oh_transformer = OneHotEncoder(drop="first")

        preprocessor = ColumnTransformer(
            [
                ("OneHotEncoder", oh_transformer, cat_features),
                ("StandardScaler", numeric_transformer, num_features),
            ]
        )

        preprocessor.fit(train_X)

        preprocessor_model_path = os.path.join(
            self.config.root_dir, self.config.preprocessor_model_name
        )
        joblib.dump(preprocessor, preprocessor_model_path)
        logger.info(f"Saved preprocessor model at {preprocessor_model_path}")

In [38]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
    data_transformation.fit_and_save_column_transformer()
except Exception as e:
    raise e

[2024-09-08 23:17:59,128: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-08 23:17:59,129: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-08 23:17:59,132: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-08 23:17:59,133: INFO: common: created directory at: artifacts]
[2024-09-08 23:17:59,133: INFO: common: created directory at: artifacts/data_transformation]
[2024-09-08 23:17:59,190: INFO: 2036974194: Splited data into training and test sets]
[2024-09-08 23:17:59,191: INFO: 2036974194: (3421, 18)]
[2024-09-08 23:17:59,191: INFO: 2036974194: (1467, 18)]
[2024-09-08 23:17:59,217: INFO: 2036974194: Saved preprocessor model at artifacts/data_transformation/preprocessor.joblib]
